In [1]:
!pip install keras

In [ ]:
!pip install tensorflow

In [3]:
import tensorflow as tf
tf.config.list_physical_devices('GPU') #  команда покажет название видеокарты, подходящей для использования при обучении нейронок

[]

In [4]:
import tensorflow as tf
tf.test.gpu_device_name() # Уточним, используется ли сейас графический процессор

''

In [6]:
import numpy as np
np.random.seed(123)  # для воспроизводимости
from keras.models import Sequential                           # Sequential - это базовый класс нейронки в керасе
from keras.layers import Dense, Dropout, Activation, Flatten  # дальше импортируем классы слоев
from keras.layers import Convolution2D, MaxPooling2D          # еще немного слоев, которые предназначены именно для работы с картинками
from keras.utils import np_utils
from keras.datasets import mnist                              # а это - классический набор картинок с рукописными цифрами, которые мы будем
                                                              # классифицировать в нашем примере

In [ ]:
# загрузим архив с фотографиями во временную папку /tmp/ и назовем его dataset.zip

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/dataset.zip

In [8]:
import shutil
import zipfile

local_zip = '/tmp/dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall("/tmp/dataset")
!mkdir dataset
!mv /tmp/dataset/cats_and_dogs_filtered/train/* dataset/
!mv /tmp/dataset/cats_and_dogs_filtered/validation/cats/* dataset/cats/
!mv /tmp/dataset/cats_and_dogs_filtered/validation/dogs/* dataset/dogs/
!rm -rf /tmp/dataset
!rm -rf /tmp/dataset.zip

In [9]:
import os

In [10]:
# Функция, которая перераспределяет файлы по выборкам

def train_test_split_images(str):
  # Создадим нужные папки
  level_1 = str + '/animals'
  os.mkdir(level_1)
  level_2_1 = level_1 + '/test'
  level_2_2 = level_1 + '/train'
  os.mkdir(level_2_1)
  os.mkdir(level_2_2)
  level_3_1 = level_2_1 + '/cats'
  level_3_2 = level_2_1 + '/dogs'
  level_3_3 = level_2_2 + '/cats'
  level_3_4 = level_2_2 + '/dogs'
  os.mkdir(level_3_1)
  os.mkdir(level_3_2)
  os.mkdir(level_3_3)
  os.mkdir(level_3_4)

# Распределим файлы
  len1 = len(os.listdir('./dataset/cats'))
  for n in range(0, len1):
    if n % 5 == 0:
      path = './dataset/animals/test/cats'
    else:
      path = './dataset/animals/train/cats'
    file = './dataset/cats/' + os.listdir('./dataset/cats')[0]
    shutil.move(file , path)

  len2 = len(os.listdir('./dataset/dogs'))
  for n in range(0, len2):
    if n % 5 == 0:
      path = './dataset/animals/test/dogs'
    else:
      path = './dataset/animals/train/dogs'
    file = './dataset/dogs/' + os.listdir('./dataset/dogs')[0]
    shutil.move(file , path)

In [11]:
train_test_split_images('./dataset')

In [ ]:
# shutil.rmtree(os.path.join('./dataset/animals')) # Удаление папки

In [ ]:
# Если нужна другая выборка, то можно просто перемешать исходную

import random

random.shuffle(data)

In [ ]:
os.listdir('./dataset/animals/train')  # Посмотрим состав папки

['cats', 'dogs']

In [ ]:
len(os.listdir('./dataset/animals/train/cats'))

1200

In [12]:
len(os.listdir('./dataset/animals/test/cats'))

300

---

---

В этой работе мы будем использовать нейросеть определенного типа - VGG19 с весами ResNet, параметры изображений для которой зададим заранее.

In [13]:
import numpy as np
np.random.seed(123)  # для воспроизводимости
from keras.models import Sequential
from keras.layers import ZeroPadding2D, Convolution2D, MaxPooling2D, Dropout, Flatten, Activation, Dense
from keras.utils import np_utils
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from keras.preprocessing.image import ImageDataGenerator

In [14]:
import os

# размер для ResNet'a
img_width, img_height = 224, 224

# корневая папка
root_dir = './dataset'  # введите сюда адрес корневой папки
# папка с тренировочными картинками
train_dir = os.path.join(root_dir, 'animals', 'train')
# и папка с тестовыми
test_dir = os.path.join(root_dir, 'animals', 'test')
# размер батча
batch_size = 16

В фотографиях, с которыми придется иметь дело нейросети, кошки и собаки могут быть абсолютно разные, сфотографированные под разными углами и с разной степенью приближения. Имея конечную тренировочную выборку, мы можем ее расширить, поворачивая и приближая имеющиеся изображения. В этом нам поможет класс [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,  # приведем значения пикселей к 0-1
    shear_range=0.2,   # максимальный угол поворота изображений
    zoom_range=0.2,    # максимальное приближение изображений
    horizontal_flip=True)  # используем повороты изображений для обогащения обучающей выборки

test_datagen = ImageDataGenerator(rescale=1. / 255)  # замер качества мы должны проводить на оригинальных изображениях, 
                                                     # поэтому тестовую выборку мы не будем вращать и приближать

Далее считаем изображения из папки, приведя их к тому размеру, с которым работает сеть VGG19. Кстати говоря, преимущество ImageDataGenerator'а в том, что картинки, разложенные по подпапкам, автоматически считаются объектами разных классов.

In [16]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator =  test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


# Задание 2. Переиспользование топологий нейронных сетей

Построим модель. Ваша задача - найти в интернете архитектуру VGG19 и описать нейросеть по этой архитектуре через keras. Рекомендую ориентироваться на архитектуру, описанную в официальном репозитории кераса на github.com. Ссылку на него здесь не приведена сознательно, поскольку большая часть работы программиста заключается в умении гуглить, и этот навык лучше тренировать сразу.

In [17]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
import cv2, numpy as np

In [24]:
def vlg_34():
    from keras import backend
    from keras.applications import imagenet_utils
    from keras.engine import training
    from keras.layers import VersionAwareLayers
    from keras.utils import data_utils
    layers = VersionAwareLayers()
    input_shape = imagenet_utils.obtain_input_shape(
        None,default_size=224,min_size=32,
        data_format=backend.image_data_format(),
        require_flatten=True,weights='imagenet')
    img_input = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    x = layers.Flatten(name='flatten')(x)
    x = layers.Dense(4096, activation='relu', name='fc1')(x)
    x = layers.Dense(4096, activation='relu', name='fc2')(x)
    imagenet_utils.validate_activation('softmax', 'imagenet')
    x = layers.Dense(1000, activation='softmax',
                        name='predictions')(x)
    model2 = training.Model(img_input, x, name='vgg19')
    model2.load_weights(data_utils.get_file(
            'vgg19_weights_tf_dim_ordering_tf_kernels.h5',
            'https://storage.googleapis.com/tensorflow/keras-applications/'
                'vgg19/vgg19_weights_tf_dim_ordering_tf_kernels.h5',
            cache_subdir='models',
            file_hash='cbe5617147190e668d6c5d5026f83318'))
    return model2
model = Sequential()
model.add(vlg_34())
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'])
model.save_weights(os.path.join(root_dir,'vgg19_weights_tf_dim_ordering_tf_kernels.h5'))

574726144/574710816 [==============================] - 5s 0us/step


In [ ]:
model = Sequential()
inputShape = (height,width,depth)
        
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x))
        
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x))
 
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x))

model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x))

model.add(layers.Flatten(name='flatten')(x))
model.add(layers.Dense(4096, activation='relu', name='fc1')(x))
model.add(layers.Dense(4096, activation='relu', name='fc2')(x))
#model.add(layers.Dense(1000, activation='softmax', name='predictions')(x))

In [19]:
# Сохраним веса модели

#model.save_weights('./dataset/vgg19_weights_tf_dim_ordering_tf_kernels.h5')

Подгрузим веса модели, подобранные экспертами до нас. Если на предыдущем этапе вы создали неправильную архитектуру, следующая ячейка упадет с ошибкой.

In [25]:
# Загрузим веса

model.load_weights(os.path.join(root_dir, 'vgg19_weights_tf_dim_ordering_tf_kernels.h5'))

Осталось добавить полносвязный слой из двух нейронов (по количеству наших классов) с функцией активации softmax, чтобы наша модель могла производить бинарную классификацию изображений.

In [26]:
# Добавим полносвязный слой из двух нейронов 

model.add(Dense(2, activation='softmax'))  # у нас стоит задача двухклассовой классификации, поэтому в выходном слое 2 нейрона

Скомпилируем и обучим сеть. Поскольку данные у нас теперь будут подаваться через генераторы, обучать модель нужно не методом fit, а методом fit_generator.

In [27]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [28]:
# для примера обучим одну эпоху
model.fit_generator(
    train_generator,
    steps_per_epoch=250 // batch_size,
    epochs=1,
    workers=3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


15/15 [==============================] - 24s 571ms/step - loss: 0.7053 - accuracy: 0.5250


# Задание 3. Обучение сети

Обучите 20 эпох сети, замеряя качество после каждой эпохи. Постройте график зависимости качества классификации от количества эпох сети. На какой эпохе качество модели превысит 0.45?

In [ ]:
scores = {}
for n_epochs in range(1, 20): 
    model.fit_generator(train_generator, steps_per_epoch=250 // batch_size, validation_data=test_generator, epochs=20, workers=3, vebrose=level_3_1))
    score = model.evaluate(train_generator, verbose=0)  # замеряем текущее качество классификации
    scores[n_epochs] = score  # сохраним текущее качество, чтобы построить график

# А теперь посмотрим, как менялось качество в процессе обучения нейронки
plt.plot(list(scores.keys()), list(map(lambda x: x[1], scores.values())))
plt.xlabel("Количество эпох")
plt.ylabel("Качество классификации")
plt.title("Качество классификации цифр в зависимости от количества эпох")
plt.show()

# Задание 4. Готовые архитектуры нейросетей

В keras описаны некоторые популярные архитектуры нейросетей. При этом их можно использовать в качестве одного из слоев другой нейросети. В частности, есть готовый класс, в котором описана VGG19 с теми весами, которые мы подгружали из файла (ImageNet).

In [ ]:
from keras.applications.vgg19 import VGG19

ready_model = Sequential()
ready_model.add(VGG19(weights='imagenet'))
ready_model.add(Dense(2, activation='softmax')) 
ready_model.compile(optimizer='sgd', loss='categorical_crossentropy')

Изучите, какие нейросети также популярны для классификации изображений. Выберите 2 архитектуры, постройте модели с их использованием, выберите наилучшую по качеству на 20 эпохах и лучшую по скорости обучения.

In [ ]:
# решение задания 4